# Porsche Competetive Scraper
## Goal: Scrape the App Store for the My Porsche App & Competitors

### Load required libraries

In [ ]:
import pandas as pd  
from matplotlib import pyplot as plt
import matplotlib.ticker as mtick

from itunes_app_scraper.scraper import AppStoreScraper
from app_store_scraper import AppStore

import datetime as dt
from tzlocal import get_localzone

import random
from pprint import pprint
import time
import os
import string
import re
import math

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import TfidfModel

import spacy
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

import pyLDAvis
import pyLDAvis.gensim_models

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
!python -m spacy download de_core_news_sm  # download spacy trained pipeline for german language

### Helper functions

In [ ]:
def load_app_list():
    """
    Read file that includes all apps that are to be scraped
    """
    df = pd.read_csv("app_list_updated.csv", sep=";")
    df["iOS_app_id"] = df["iOS_app_id"].str.replace("id", "").astype("int")
    return df


def get_general_app_info(app_list_df):
    """
    Take loaded app list as input and return meta information from the
    app store about every app (indicated by app id) in the list.
    """
    scraper = AppStoreScraper()
    app_ids = app_list_df["iOS_app_id"]
    app_meta_info = list(scraper.get_multiple_app_details(app_ids, country="de"))
    app_meta_info_df = pd.DataFrame(app_meta_info)
    return app_meta_info_df


def get_reviews(app_list_df, country_code="de"):
    """
    This function returns a data frame including scraped reviews from the App store.
    Parameters:
    app_list_df - data frame including the app id and name (as displayed in the app store url).
    country_code - 2-digit country code specifiying from which local app store the reviews should be scraped. 
    """
    app_names = app_list_df["iOS_app_name"]
    app_ids = app_list_df["iOS_app_id"]
    
    # gather reviews for each app
    for app_name, app_id in zip(app_names, app_ids):
        
        # get current time
        start_time = dt.datetime.now(tz=get_localzone())
        time_format = "%m/%d/%y - %T %p"
        
        # print starting of scraping app 1,...,n
        print("----"*20)
        print("----"*20)
        print(f"x-x-x-x-x-x {app_name} started at {start_time.strftime(time_format)}")
        
        # initialize AppStore object
        app_ = AppStore(country=country_code, app_name=app_name, app_id=app_id)
        # get ALL reviews available for the app in the countries app store
        app_.review(sleep=random.randint(5, 10))
        
        cur_reviews = app_.reviews
        
        # add meta information to review table
        for review in cur_reviews:
            review["app_name"] = app_name
            review["app_id"] = app_id
            
        # print ending of scraping app 1,...,n
        print(f"Done scraping {app_name}. Scraped {app_.reviews_count} reviews.")
        
        # convert to data frame and write to csv
        review_df = pd.DataFrame(cur_reviews)
        review_df.to_csv("reviews_" + app_name + ".csv", index=False, encoding="utf-8-sig")
        
        # provide finishing message
        end_time = dt.datetime.now(tz=get_localzone())
        print(f"Wrote {app_name} reviews to csv at {end_time.strftime(time_format)}.")
        print(f"Required time for {app_name}: {end_time-start_time}")
        
        # wait a few seconds before starting with the next app
        time.sleep(random.randint(5, 10))

        
def read_and_merge_reviews():
    """
    Returns a data frame that contains all the merged reviews scraped from the app store.
    """
    all_files = os.listdir()
    review_files = [review for review in all_files if "reviews_" in review]
    df = pd.concat(map(pd.read_csv, review_files)).reset_index()
    return df


def add_labels(x, y):
    """
    Add centered labels to plot
    """
    for i in range(len(x)):
        plt.text(i, y[i], y[i], ha = 'center')


def plot_bar_from_group(grouped_data, title, xlabel, ylabel):
    """
    Plot the count of each grouped class in a bar graph.
    """
    cust_dict = {
        "group_label": list(grouped_data.index),
        "count": list(grouped_data.values)
    }
    
    df = pd.DataFrame(cust_dict)
    df = df.sort_values(by="count").reset_index()
    
    plt.figure(figsize=(10,6))
    plt.bar(df["group_label"], df["count"])
    add_labels(df["group_label"], df["count"])
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.xticks(rotation=45)
    
    plt.savefig('average_rating_written_feeback_only.png', bbox_inches = "tight")
    plt.show()   
    
    
def convert_to_datetime(date_col):
    """
    Return converted datetime object based on date_col string input.
    """
    out = []
    for date in date_col:
        converted_date = dt.datetime.strptime(date, "%Y-%m-%d %H:%M:%S")
        out.append(converted_date)
    
    return out


def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    """
    This function applies lemmatization to the list of reviews passed as input. 
    Lemmatization reduces words to its stem i.e. talked, talking, talk -> talk.
    Additionally, the nlp() basic preprocessing is applied. i.e. lower case, removal of numbers/special characters.
    
    Returns a cleaned/lemmatized list of reviews.
    """
    nlp = spacy.load("de_core_news_sm", disable=["parser", "ner"])  
    texts_out = []
    for text in texts:
        document = nlp(text)
        # iterate over each token in the document
        new_text = []
        for token in document:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        merged_lemma_sentence = " ".join(new_text)
        texts_out.append(merged_lemma_sentence)
    return texts_out


def generate_word_tokens(texts):
    """
    Generates a list of seperated words for each review. Additonally, applies lower case to remove semantic redundancy.
    """
    processed_reviews = []
    for text in texts:
        new_text = gensim.utils.simple_preprocess(text, deacc=True)
        processed_reviews.append(new_text)
    return processed_reviews


def create_dict_and_corpus(data_words):
    """
    Generates: 1. A word dictionary 
               2. A corpus
    """
    # create translation dictionary
    id2word = corpora.Dictionary(data_words)
    # create corpus
    corpus = []
    for review in data_words:
        new_review = id2word.doc2bow(review)
        corpus.append(new_review)
        
    return id2word, corpus



def compute_coherence_score(k_start, k_max, steps, id2word, corpus, text):
    """
    Estimate a LDA model for the range of topics and evaluate its coherence scores.
    """
    coherence_scores = []
    models = []
    for num_topics in range(k_start, k_max, steps):
        cur_lda = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                                 id2word=id2word,
                                                 num_topics=num_topics,
                                                 random_state=42,
                                                 update_every=1,
                                                 chunksize=100,
                                                 passes=10,
                                                 alpha="auto")
        models.append(cur_lda)
        coherence_model = CoherenceModel(model=cur_lda, texts=text, dictionary=id2word, coherence="c_v")
        coherence_scores.append(coherence_model.get_coherence())
    return models, coherence_scores



def remove_junk(texts, stops):
    """
    This function takes a review as input and does the following:
    1.) remove german stopwords
    2.) remove punctiation
    3.) remove numbers
    4.) remove unnessary white space
    """
    text = re.sub(r"\d+", "", texts)
    words = texts.split()
    final = []
    for word in words:
        # remove stop words
        if word not in stops:
            final.append(word)
    final = " ".join(final)
    # remove punctuation
    final = final.translate(str.maketrans("", "", string.punctuation))
    # remove numbers
    final = "".join([i for i in final if not i.isdigit()])
    # remove double whitespace
    while "  " in final:
        final = final.replace("  ", " ")
    return final


def clean_document(docs):
    """
    This function applies the remove_junk function to a list of reviews
    """
    stops = stopwords.words("german")
    final = []
    for doc in docs:
        clean_doc = remove_junk(doc, stops)
        final.append(clean_doc)
    return final


def format_topics_sentences(lda_model, corpus, review_df):
    """
    This function returns the review_df data frame with the additinal feaatures of:
    1.) Dominant topic
    2.) Perc. Contribution of Dominant Topic
    3.) Top Keywords Associated to Topic i
    """
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(lda_model[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = lda_model.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']


    sent_topics_df = pd.concat([sent_topics_df, review_df], axis=1)
    return(sent_topics_df)


def investigate_top_perc_contribution(df):
    """
    Takes the review data frame that includes dominant topic as an input.
    Returns one sentence for each topic that has the highest percentual contibution.
    Thus, gives examples of what sentencs that describe the certain topic look like.
    """
    output = pd.DataFrame()
    grouped_df = df.groupby("Dominant_Topic")
    
    for i, group in grouped_df:
        output = pd.concat([output, group.sort_values(["Perc_Contribution"], ascending=False).head(1)], axis=0)
        
    output.reset_index(drop=True, inplace=True)
    return output


        

### Apply all relevant functions and gather review data

In [ ]:
# load app list
app_list_df = load_app_list()

In [ ]:
app_list_df

In [ ]:
# get meta app information
meta_app_df = get_general_app_info(app_list_df)

In [ ]:
meta_app_df

In [ ]:
# append app name to meta data
meta_app_df["app_name"] = app_list_df["app_name"]

In [ ]:
# convert rating to log scale
meta_app_df["log_userRatingCount"] = meta_app_df["userRatingCount"].apply(lambda x: math.log(x))

In [ ]:
# plot app meta information for global market
fig = plt.figure(figsize=(10,6), dpi=1200)
ax = fig.add_subplot(111)
ax2 = ax.twinx()
width = 0.3
x = meta_app_df["app_name"]
meta_app_df["userRatingCount"].plot(kind='bar', color='red', ax=ax, width=width, position=1, label="Number of reviews")
meta_app_df["averageUserRating"].plot(kind='bar', color='blue', ax=ax2, width=width, position=0, label="Average Rating")
ax.set_ylabel('Number of reviews')
ax2.set_ylabel('Average Rating')
ax.set_xlabel("Application")
#ax.legend(loc="upper left")
#ax2.legend(loc="upper right")


ax.set_xticklabels(x, rotation=45)

rects_one = ax.patches
rects_two = ax2.patches

# add labels for ax 1
for rect, value in zip(rects_one, meta_app_df["userRatingCount"]):
    height = rect.get_height()
    ax.text(rect.get_x() + rect.get_width() / 2, height + 5, value, ha="center", va="bottom", size=7)
    

for rect_2, value_2 in zip(rects_two, meta_app_df["averageUserRating"]):
    height_2 = rect_2.get_height()
    ax2.text(rect_2.get_x() + rect_2.get_width() / 2, height_2 , str(round(value_2, 1)), ha="center", va="bottom", size=7)

plt.savefig('aggregate_valence_volume_landingpage.png', bbox_inches = "tight")
plt.show()

In [ ]:
# scrape and write reviews to csv in working directory (Initial Scraping Date: 23.06.2022)
get_reviews(app_list_df, country_code="de")

In [ ]:
# load and merge all scraped reviews
review_df = read_and_merge_reviews()

In [ ]:
review_df.head()

### Exploratory Data Analysis

How many reviews are available in the german App Store per application?

In [ ]:
# calculate number of reviews per application
plt.rcParams["figure.dpi"] = 1200
review_count_by_app = review_df.groupby("app_name")["review"].size()

# plot number of reviews per application
plot_bar_from_group(review_count_by_app,
                   xlabel="App Name",
                   ylabel="Number of reviews",
                   title="")


What is the average rating per app?
<p>Important finding: Average ratings and number of reviews displayed in the App Store represent global submissions.
However, when we narrow down to the german market only, we obtain a differrent picture.
The number and average rating of the My Porsche App was verified by visting the German App Store and manually scanning the reviews. Thus, for all of the apps, we observe a worse satisfaction level in Germany, compared to the global perception.

In [ ]:
# calculate average rating by application
average_rating_by_app = review_df.groupby("app_name")["rating"].mean().round(1)

# plot average rating per application
plot_bar_from_group(average_rating_by_app,
                   title="",
                   xlabel="App Name",
                   ylabel="Average Rating")

How are the ratings distributed by app?

In [ ]:
# plot rating distribution by app
plt.rcParams["figure.dpi"] = 1200
ax = (review_df[["app_name", "rating"]].groupby(['app_name','rating'])
   .value_counts().unstack('rating').plot.bar(figsize=(10,5), rot=45))
ax.set_xlabel("App")
ax.set_ylabel("Number of reviews")
ax.set_title("Distribution of ratings by app")
plt.show()

How many reviews did users submit over time?

In [ ]:
# convert date feature to datetime object
review_df["date"] = convert_to_datetime(review_df["date"])

In [ ]:
# add a year-month column
review_df["year_month"] = review_df["date"].dt.to_period("M")

In [ ]:
# Plot monthly number of reviews by app
plt.rcParams["figure.dpi"] = 1200
ax = (review_df[["app_name", "year_month"]].groupby(['app_name','year_month'])
   .value_counts().unstack('app_name').plot(figsize=(10,6), rot=45, style=".-"))
ax.set_xlabel("Time")
ax.set_ylabel("Number of reviews")
plt.legend(title="Application")
plt.savefig("development_of_review_volume_over_time.png", bbox_inches="tight")
plt.show()

In [ ]:
# Get minimum date for first reviw with written feedback per app
review_df.groupby("app_name")["date"].min()

In [ ]:
# get launch difference between first written review and scraping data
scraping_date = "2022-06-23"
scraping_date = dt.datetime.strptime(scraping_date, "%Y-%m-%d")
min_date_by_app = review_df.groupby("app_name")["date"].min()
print(min_date_by_app)
days_since_first_review = []
for date in min_date_by_app:
    date_diff = (scraping_date - date).days
    days_since_first_review.append(date_diff)
    
print(days_since_first_review)

In [ ]:
print(f"My Porsche Reviews per day: {round(218/253, 1)}")
print(f"My BMW Reviews per day: {round(58699/695, 1)}")
print(f"My Audi Rviews per day: {round(16442/4029, 1)}")
print(f"MINI Reviews per day: {round(6575/695, 1)}")
print(f"FordPass Reviews per day: {round(16472/2065, 1)}")
print(f"Mercedes Me Reviews per day: {round(83176/702, 1)}")
print(f"Volkswagen We Connect Reviews per day: {round(30113/3130, 1)}")

## Text preprocessing

In [ ]:
# sample equal number of reviews per app (with same rating weight as observed in full set)
review_subset = review_df.groupby("app_name").apply(lambda x: x.sample(80, random_state=666, weights=review_df.groupby('app_name')["rating"].transform('count'))).reset_index(drop=True)

In [ ]:
review_subset.head()

In [ ]:
# compare average rating with average rating of full data
review_subset.groupby("app_name")["rating"].mean().round(2)

In [ ]:
# verify 560 total observations (80*n_brands)
review_subset.shape

In [ ]:
# apply lemmatization (reduce word to its root-stem)
lemmatized_reviews = lemmatization(review_subset["review"])  # exchange review_subset with review_df["review"] to get full data as model input
# apply custom text cleaning
lemmatized_reviews = clean_document(lemmatized_reviews) 

In [ ]:
# tokenization
data_words = generate_word_tokens(lemmatized_reviews)

In [ ]:
data_words[0][0:10]

In [ ]:
# create index to word dictionary (required for LDA modeling)
id2word, corpus = create_dict_and_corpus(data_words)

In [ ]:
# List of tuples: (i, j) -> i = index of word in dictionary, j = frequency of word at index i
print(corpus[0][0:10])
print(f"Word of first tuple: {id2word[[0][:1][0]]}")

## Additional Preprocessing - TF-IDF

In [ ]:
# create new id2word dictionay and corpus based on bi/trigram tokenized data
id2word_iter = corpora.Dictionary(data_words)
texts_iter = data_words
corpus_iter = [id2word_iter.doc2bow(text) for text in texts_iter]

In [ ]:
# initialize TF-IDF object (goal: exclude overly-used/non specific words)
tfidf = TfidfModel(corpus_iter, id2word_iter)

low_value = 0.05
words = []
words_missing_in_tfidf = []
for i in range(0, len(corpus_iter)):
    bow = corpus_iter[i]
    low_value_words = []
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words + words_missing_in_tfidf
    for item in drops:
        words.append(id2word_iter[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids]
    
    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus_iter[i] = new_bow

In [ ]:
# get unique list of words to be filtered out
unique_words_filter_out = set(words)

In [ ]:
print(unique_words_filter_out)

In [ ]:
# remove threhold words
removed_tfidf_reviews = []
for review in data_words:
    cur_review = review
    updated_review = [word for word in cur_review if word not in unique_words_filter_out]
    removed_tfidf_reviews.append(updated_review)

In [ ]:
len(removed_tfidf_reviews)

In [ ]:
print(removed_tfidf_reviews[0])

In [ ]:
# filter out custom removal words (Iterative Process based on earlier iterations of the LDA Topic Modeling Output)
custom_removal_words = ["zb", "danke", "etc", "wohl", "warum", "einfach", "erst", "wirklich", "somit", "groß", "dabei", "usw", "bitte", "tun", "allerdings", "ganz", "standheizung","fahrtenbuch" ,"ja", "sehen", "lange", "alt"]
reviews_cleaned = []
for review in removed_tfidf_reviews:
    new_review = [word for word in review if word not in custom_removal_words]
    reviews_cleaned.append(new_review)

In [ ]:
print(reviews_cleaned[0])

In [ ]:
len(removed_custom_words)

In [ ]:
# currently with reviews_cleaned
id2word_updated = corpora.Dictionary(reviews_cleaned)  # to revert to initial replace with removed_tfidf_reviews
corpus_updated = [id2word_updated.doc2bow(text) for text in reviews_cleaned]  # to revert to initial replace with removed_tfidf_reviews

In [ ]:
# estimate LDA with abitrary number of topic to see if it works on processed data. In this case 8
lda_model_after_tfidf_removal = gensim.models.ldamodel.LdaModel(corpus=corpus_updated,
                                                               id2word=id2word_updated,
                                                               num_topics=8,
                                                               random_state=42,
                                                               update_every=1,
                                                               chunksize=100,
                                                               passes=10,
                                                               alpha="auto")

In [ ]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim_models.prepare(lda_model_after_tfidf_removal, corpus_updated, id2word_updated, mds="mmds", R=30)

In [ ]:
# Calculate coherence score:
coherence_model_lda = CoherenceModel(model=lda_model_after_tfidf_removal, 
                                     texts=data_words, 
                                     dictionary=id2word_updated, 
                                     coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

## Find optimal number of topics

In [ ]:
# initialize topic space
k_start = 2
k_max = 9
steps = 1

In [ ]:
start = time.time()
models, coherence_scores = compute_coherence_score(k_start=k_start, 
                                                   k_max=k_max, 
                                                   steps=steps, 
                                                   id2word=id2word_updated, 
                                                   corpus=corpus_updated,
                                                   text=data_words)
end = time.time()
print(end-start)

In [ ]:
# plot coherence scores
x = list(range(k_start, k_max, steps))
plt.figure(figsize=(10,6))
plt.plot(x, coherence_scores)
plt.xlabel("Number of Topics")
plt.ylabel("Coherence Score")
plt.savefig('coherence_score.png', bbox_inches = "tight")
plt.show()

In [ ]:
# get coherence scores
for topic_number, coherence in zip(x, coherence_scores):
    print("Num Topics =", topic_number, " has Coherence Value of", round(coherence, 4))

The coherence score is maximized at num_topics = 6.

## Final LDA Model with n_topics = 6

In [ ]:
final_lda = gensim.models.ldamodel.LdaModel(corpus=corpus_updated,
                                                               id2word=id2word_updated,
                                                               num_topics=6,
                                                               random_state=42,
                                                               update_every=1,
                                                               chunksize=100,
                                                               passes=10,
                                                               alpha="auto")

In [ ]:
# IMPORTANT NOTE: LDAvis Topic numbering does NOT correspond to actual Topic numbering (see below for accurate.)
# This is just an interactive view on how destinguishable the topics are
pyLDAvis.enable_notebook()
pyLDAvis.gensim_models.prepare(final_lda, corpus_updated, id2word_updated, mds="mmds", R=30)

In [ ]:
# Word Cloud of keywords in Topics (https://www.machinelearningplus.com/nlp/topic-modeling-visualization-how-to-present-results-lda-models/)
# 1. Wordcloud of Top N words in each topic
from wordcloud import WordCloud, STOPWORDS
import matplotlib.colors as mcolors

cols = [color for name, color in mcolors.TABLEAU_COLORS.items()]  # more colors: 'mcolors.XKCD_COLORS'

cloud = WordCloud(stopwords=stopwords.words("german"),
                  background_color='white',
                  width=2500,
                  height=1800,
                  max_words=10,
                  colormap='tab10',
                  color_func=lambda *args, **kwargs: cols[i],
                  prefer_horizontal=1.0)

topics = final_lda.show_topics(formatted=False)

fig, axes = plt.subplots(2, 3, figsize=(13,13), sharex=True, sharey=True)

for i, ax in enumerate(axes.flatten()):
    fig.add_subplot(ax)
    topic_words = dict(topics[i][1])
    cloud.generate_from_frequencies(topic_words, max_font_size=300)
    plt.gca().imshow(cloud)
    plt.gca().set_title('Topic ' + str(i), fontdict=dict(size=16))
    plt.gca().axis('off')


plt.subplots_adjust(wspace=0, hspace=0)
plt.axis('off')
plt.margins(x=0, y=0)
plt.tight_layout()
plt.show()

In [ ]:
# https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#19findthemostrepresentativedocumentforeachtopic
# Visalize top keywords

from collections import Counter
topics = final_lda.show_topics(formatted=False)
data_flat = [w for w_list in reviews_cleaned for w in w_list]  # replace "reviews_cleaned" with removed_tfidf_reviews to revert to original layout
counter = Counter(data_flat)

out = []
for i, topic in topics:
    for word, weight in topic:
        out.append([word, i , weight, counter[word]])

df = pd.DataFrame(out, columns=['word', 'topic_id', 'importance', 'word_count'])        

# Plot Word Count and Weights of Topic Keywords
fig, axes = plt.subplots(2, 3, figsize=(30,18), sharey=True, dpi=600)
cols = [color for name, color in mcolors.TABLEAU_COLORS.items()]
for i, ax in enumerate(axes.flatten()):
    ax.bar(x='word', height="word_count", data=df.loc[df.topic_id==i, :], color=cols[i], width=0.5, alpha=0.3, label='Word Count')
    ax_twin = ax.twinx()
    ax_twin.bar(x='word', height="importance", data=df.loc[df.topic_id==i, :], color=cols[i], width=0.2, label='Weights')
    ax.set_ylabel('Word Count', color=cols[i])
    ax_twin.set_ylim(0, 0.030); ax.set_ylim(0, 200)
    ax.set_title('Topic: ' + str(i), color=cols[i], fontsize=16)
    ax.tick_params(axis='y', left=False)
    ax.set_xticklabels(df.loc[df.topic_id==i, 'word'], rotation=30, horizontalalignment= 'right')
    ax.legend(loc='upper left'); ax_twin.legend(loc='upper right')

fig.tight_layout(w_pad=2)    
fig.suptitle('', fontsize=22, y=1.05)
             
plt.savefig("LDA_wordfrequencies_and_weights.png", bbox_inches="tight")           
plt.show()

### Naming Topics (Subjective but whatever :D):
Takes into consideration the wordcloud/frequency as well as the LDAvis output that shows top 30 relevant keywords.
- <b> Topic 0 </b>
<p>Missing Updates
- <b> Topic 1 </b>
<p> Connectivity Issues
- <b> Topic 2 </b>
<p> Time-based Support    
- <b> Topic 3 </b>
<p> Legacy Systems
- <b> Topic 4 </b>
<p> Application Error
- <b> Topic 5 </b>
<p>Legacy Connectivity
- <b> Topic 6 </b>
<p> Useability and Activation
- <b> Topic 7 </b>
<p> Loading and Login Service    
- <b> Topic 8 </b>
<p> Feature Availibility
- <b> Topic 9 </b>
<p> Access Error Support Request
    
-------- UPDATED TOPICS WITH REMOVED CUSTOM WORDS (n_num=6)
- <b> Topic 0 </b>
<p>Error Message for Sign-Up & Update Service
- <b> Topic 1 </b>
<p> Customizability
- <b> Topic 2 </b>
<p> Slow Updates   
- <b> Topic 3 </b>
<p> Connectivity Problems
- <b> Topic 4 </b>
<p> Time-based Charging
- <b> Topic 5 </b> 
<p> Missing Feature 

## Find the dominant topic in each review

In [ ]:
df_with_topics = format_topics_sentences(final_lda, corpus_updated, review_subset)  # exchange review_subset with review_df for entire data!

In [ ]:
name_dictionary = {0: "Missing Update",
                  1:  "Connectivity Issue",
                  2:  "Time-based Support",
                  3:  "Legacy Systems",
                  4:  "Application Error",
                  5:  "Legacy Connectivity", 
                  6:  "Usability & Activation",
                  7:  "Loading & Login Service",
                  8:  "Feature Availibility",
                  9:  "Access Error Support Request"}


name_dictionary_new = {0: "Error Message for Signup & Update Service",
                  1:  "Customizability",
                  2:  "Slow Updates",
                  3:  "Connectivity Problems",
                  4:  "Time-based Charging",
                  5:  "Missing Feature"} 
                    
    

def add_topic_name(name_dictionary, df):  
    topic_name = []
    for topic_number in df["Dominant_Topic"]:
        topic_name.append(name_dictionary.get(topic_number))
    
    topic_name = pd.Series(topic_name)
    return topic_name
    
    
df_with_topics["Topic_Name"] = add_topic_name(name_dictionary_new, df_with_topics)   # replace name_dictionary_new with name_dictionary to revert to old results

In [ ]:
df_with_topics.head()

In [ ]:
set(df_with_topics["Dominant_Topic"])  # 0 -> Topic 1 etc. Apparently topic 2 is NOT the most dominant in any case.

## Plot distribution of Topics by App

In [ ]:
# plot number of topics by app
ax = (df_with_topics[["app_name", "Topic_Name"]].groupby(['app_name','Topic_Name'])
   .value_counts().unstack('Topic_Name').plot.bar(figsize=(10,8), rot=45))
ax.set_xlabel("Application")
ax.set_ylabel("Number of Topic occurences")
ax.set_title("")
plt.savefig("Distribution_of_topics_by_app.png", bbox_inches="tight")
plt.show()

In [ ]:
(df_with_topics[["app_name", "Topic_Name"]].groupby(["app_name", "Topic_Name"]).value_counts() / 80*100)

In [ ]:
# get percentages
ax = (df_with_topics[["app_name", "Topic_Name"]].groupby(["app_name", "Topic_Name"]).value_counts() / 80*100).unstack("Topic_Name").plot.bar(figsize=(10,8), rot=45)
ax.set_xlabel("Application")
ax.set_ylabel("Percentage of topic occurence within application")
ax.set_title("")
fmt = '%.0f%%' # Format you want the ticks, e.g. '40%'
yticks = mtick.FormatStrFormatter(fmt)
ax.yaxis.set_major_formatter(yticks)
plt.legend(title="Topic Name")
plt.savefig("Percentage_of_topics_by_app.png", bbox_inches="tight")
plt.show()

### What is the average rating by app and Topic. Do some apps perform better within certain categories?


In [ ]:
# start with getting averge rating by topic for porsche
porsche_topics_subset = df_with_topics[df_with_topics["app_name"] == "my-porsche"]

# plot average rating by topic
porsche_topics_subset.groupby("Topic_Name")["rating"].mean().plot(kind="bar")

In [ ]:
# Get average rating by topic per app
ax = df_with_topics.groupby(["app_name", "Topic_Name"])["rating"].mean().unstack("Topic_Name").plot(kind="bar", figsize=(10,8), rot=45)
ax.set_xlabel("App")
ax.set_ylabel("Average Rating")
ax.set_title("")
plt.legend(title="Topic Name")
plt.savefig("averageRating_of_topics_by_app.png", bbox_inches="tight")
plt.show()

In [ ]:
# output percentual topic probabilities per review (one list entry = 1 review)
for i, row in enumerate(final_lda[corpus_updated]):
    row = sorted(row, key=lambda x:x[1], reverse=True)
    print(row)

In [ ]:
# extract top sentences that have a high percentual contribution in the corresponding topic
top_sentences = investigate_top_perc_contribution(df_with_topics)

In [ ]:
# display exemplary sentneces for each topic (+1 index)
for index, review in enumerate(top_sentences["review"]):
    print(f"Topic {index+1}:\n {review}")
    print("----------")

## Try TF-IDF and KMeans approach to cluster short text into topics

In [ ]:
# apply cleaning functions to raw text reviews
clean_reviews = clean_document(lemmatized_reviews)  # basically redundant as already performed earlier

In [ ]:
vectorizer = TfidfVectorizer(lowercase=True,
                            max_features=100,
                            max_df=0.75,  # words that occur at greater than 75% in the corpus are ignored
                            min_df=5,  # if a word does not occur at least 5 times, ignore it
                            ngram_range=(1,3))

In [ ]:
# apply TF-IDF
vectors = vectorizer.fit_transform(clean_reviews)
feature_names = vectorizer.get_feature_names()
dense = vectors.todense()
denselist = dense.tolist()

all_keywords = []
for description in denselist:
    x = 0
    keywords = []
    for word in description:
        if word > 0:
            keywords.append(feature_names[x])
        x += 1
    all_keywords.append(keywords)

In [ ]:
print(all_keywords[0])
print("--------")
print(clean_reviews[0])  # altered for TF-IDF approach --> Keywords of the revieew

## Cluster the list of keywords based on TF-IDF by KMeans

In [ ]:
# find optimal number of clusters (initially using the elbow criterion)
distortions = []
K = range(1, 20)
for k in K:
    kmean_model = KMeans(n_clusters=k, init="k-means++", max_iter=100, n_init=1)
    kmean_model.fit(vectors)
    distortions.append(kmean_model.inertia_)

In [ ]:
# plot distortions = sum of squared distance between each observation vector
plt.figure(figsize=(16,10))
plt.plot(list(K), distortions, "bx-")
plt.xlabel("Number of Clusters (k)")
plt.ylabel("Distortion")
plt.title("Elbow Method showing the optimal number of clusters k")
plt.show()

Based on this we can conclude that K = 10 is the seems appropriate.

In [ ]:
# fit final Kmeans with k = 10
final_kmeans = KMeans(n_clusters=10,
                     init="k-means++",
                     max_iter=100,
                     n_init=1)
final_kmeans.fit(vectors)

In [ ]:
# extract information
order_centroids = final_kmeans.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()

In [ ]:
# print topic keywords that are associated with each cluster
for i in range(8):
    print(f"Cluster {i}")
    print("\n")
    for ind in order_centroids[i, :15]:
        print(" %s" %terms[ind],)
    print("\n")
    print("\n")

In comparison to the LDA approach, topic modelling with TFIDF and KMeans does not seem to yield interpretable results. Furthmore, we observe a even greater topic overlap in comparison to the LDA soltution. Thus, we consider the LDA approach more feasible for the final interpretation.

In [ ]:
# TBC
print("------")